In [2]:
#Stacking with a Neural Network

In [4]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
from sklearn.datasets import fetch_california_housing
 
#From within an ipython notebook
%matplotlib inline
 
cali_housing = fetch_california_housing()
 
X = cali_housing.data
y = cali_housing.target

bins = np.arange(6)
binned_y = np.digitize(y, bins)

In [6]:
from sklearn.model_selection import train_test_split
X_train_prin, X_test_prin, y_train_prin, y_test_prin = train_test_split(X, y,test_size=0.2,stratify=binned_y,random_state=7)

binned_y_train_prin = np.digitize(y_train_prin, bins)

X_1, X_stack, y_1, y_stack = train_test_split(X_train_prin,y_train_prin,test_size=0.33,stratify=binned_y_train_prin,random_state=7 )

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
 
mlp_pipe = Pipeline(steps=[('scale', StandardScaler()), ('neural_net', MLPRegressor())])
 
 
param_grid = {'neural_net__alpha': [0.02,0.01,0.005],
               'neural_net__hidden_layer_sizes' : [(50,50,50)],
               'neural_net__activation': ['relu'],
               'neural_net__solver' : ['adam']
               }
 
neural_net_gs = GridSearchCV(mlp_pipe, param_grid = param_grid,cv=3, n_jobs=-1)
neural_net_gs.fit(X_1, y_1)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('neural_net', MLPRegressor())]),
             n_jobs=-1,
             param_grid={'neural_net__activation': ['relu'],
                         'neural_net__alpha': [0.02, 0.01, 0.005],
                         'neural_net__hidden_layer_sizes': [(50, 50, 50)],
                         'neural_net__solver': ['adam']})

In [9]:
neural_net_gs.best_params_

{'neural_net__activation': 'relu',
 'neural_net__alpha': 0.01,
 'neural_net__hidden_layer_sizes': (50, 50, 50),
 'neural_net__solver': 'adam'}

In [12]:
neural_net_gs.best_score_

0.7839312445650505

In [14]:
nn_best = neural_net_gs.best_estimator_
import pickle
 
f = open('nn_best.save', 'wb')
pickle.dump(nn_best, f, protocol = pickle.HIGHEST_PROTOCOL)
f.close()

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
 
 
param_grid = {'learning_rate': [0.1,0.05,0.03,0.01],
               'loss': ['huber'],
               'max_depth': [5,7,10],
               'max_features': [0.4,0.6,0.8,1.0],
               'min_samples_leaf': [2,3,5],
               'n_estimators': [100],
               'warm_start': [True], 'random_state':[7]
               }
 
boost_gs = RandomizedSearchCV(GradientBoostingRegressor(), param_distributions = param_grid,cv=3, n_jobs=-1,n_iter=25)
boost_gs.fit(X_1, y_1)

RandomizedSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_iter=25,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.05, 0.03,
                                                          0.01],
                                        'loss': ['huber'],
                                        'max_depth': [5, 7, 10],
                                        'max_features': [0.4, 0.6, 0.8, 1.0],
                                        'min_samples_leaf': [2, 3, 5],
                                        'n_estimators': [100],
                                        'random_state': [7],
                                        'warm_start': [True]})

In [18]:
boost_gs.best_score_

0.8239187597559422

In [20]:
boost_gs.best_params_

{'warm_start': True,
 'random_state': 7,
 'n_estimators': 100,
 'min_samples_leaf': 5,
 'max_features': 0.6,
 'max_depth': 7,
 'loss': 'huber',
 'learning_rate': 0.1}

In [22]:
gbt_inst = GradientBoostingRegressor(**{'learning_rate': 0.1,
 'loss': 'huber',
 'max_depth': 10,
 'max_features': 0.4,
 'min_samples_leaf': 5,
 'n_estimators': 4000,
 'warm_start': True, 'random_state':7}).fit(X_1, y_1)

In [24]:
def pickle_func(filename, saved_object):
    import pickle
 
    f = open(filename, 'wb')
    pickle.dump(saved_object, f, protocol = pickle.HIGHEST_PROTOCOL)
    f.close()
     
    return None
 
pickle_func('grad_boost.save', gbt_inst)

In [33]:
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distribution
param_dist = {
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0],
    'oob_score': [True, False],
    'estimator__min_samples_leaf': [4, 5],
    'n_estimators': [20]
}

# Initialize the single estimator (GradientBoostingRegressor)
single_estimator = GradientBoostingRegressor(
    learning_rate=0.1,
    loss='huber',
    max_depth=10,
    max_features=0.4,
    n_estimators=20,
    warm_start=True,
    random_state=7
)

# Initialize the ensemble estimator (BaggingRegressor)
ensemble_estimator = BaggingRegressor(estimator=single_estimator)

# Perform RandomizedSearchCV
pre_gs_inst_bag = RandomizedSearchCV(
    ensemble_estimator,
    param_distributions=param_dist,
    cv=3,
    n_iter=5,
    n_jobs=-1
)

# Fit the model
pre_gs_inst_bag.fit(X_1, y_1)

# Output the best parameters and best score
print("Best parameters found: ", pre_gs_inst_bag.best_params_)
print("Best score achieved: ", pre_gs_inst_bag.best_score_)


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_bagging.py:1319: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_bagging.py:1319: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(


Best parameters found:  {'oob_score': True, 'n_estimators': 20, 'max_samples': 1.0, 'max_features': 1.0, 'estimator__min_samples_leaf': 5}
Best score achieved:  0.7792412131267943


In [34]:
pre_gs_inst_bag.best_score_

0.7792412131267943

In [37]:
pickle_func('bag_gbm.save', pre_gs_inst_bag.best_estimator_)

In [39]:
def pickle_load_func(filename):
    f = open(filename, 'rb')
    to_return = pickle.load(f)
    f.close()
    
    return to_return

neural_net = pickle_load_func('nn_best.save')
gbt = pickle_load_func('grad_boost.save')
bag_gbm = pickle_load_func('bag_gbm.save')

In [41]:
def handle_X_set(X_train_set_in):
    X_train_set = X_train_set_in.copy()
    
    y_pred_nn = neural_net.predict(X_train_set)
    y_pred_gbt = gbt.predict(X_train_set)
    y_pred_bag = bag_gbm.predict(X_train_set)
    
    
    preds_df = pd.DataFrame(columns = ['nn', 'gbt','bag'])

    preds_df['nn'] = y_pred_nn
    preds_df['gbt'] = y_pred_gbt
    preds_df['bag'] = y_pred_bag
 
    return preds_df

def predict_from_X_set(X_train_set_in):
    X_train_set = X_train_set_in.copy()    
    return final_etr.predict(handle_X_set(X_train_set)) 

preds_df = handle_X_set(X_stack)
print (preds_df.corr())

           nn       gbt       bag
nn   1.000000  0.956422  0.962307
gbt  0.956422  1.000000  0.981472
bag  0.962307  0.981472  1.000000


In [43]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV
 
param_dist = {'max_features' : ['sqrt','log2',1.0],
  'min_samples_leaf' : [1, 2, 3, 7, 11],
  'n_estimators': [50, 100],
  'oob_score': [True, False]}
 
pre_gs_inst = RandomizedSearchCV(ExtraTreesRegressor(warm_start=True,bootstrap=True,random_state=7),
  param_distributions = param_dist,
  cv=3,
  n_iter = 15,random_state=7)
 
pre_gs_inst.fit(preds_df.values, y_stack)

pre_gs_inst.best_params_

{'oob_score': False,
 'n_estimators': 100,
 'min_samples_leaf': 11,
 'max_features': 1.0}

In [45]:
final_etr = ExtraTreesRegressor(**{'max_features': 1.0,
  'min_samples_leaf': 11,
  'n_estimators': 3000,
  'oob_score': False, 'random_state':7}).fit(preds_df.values, y_stack)

In [47]:
from sklearn.model_selection import cross_val_score

cross_val_score(final_etr, preds_df.values, y_stack, cv=3).mean()

0.824319830463263

In [49]:
# Predict using the trained model
y_pred = pre_gs_inst_bag.predict(X_test_prin)

# Import necessary metrics
from sklearn.metrics import r2_score, mean_absolute_error

# Calculate and print R-squared
print("R-squared:", r2_score(y_test_prin, y_pred))

# Calculate and print Mean Absolute Error (MAE)
print("MAE:", mean_absolute_error(y_test_prin, y_pred))

# Calculate and print Mean Absolute Percentage Error (MAPE)
mape = (np.abs(y_test_prin - y_pred) / y_test_prin).mean()
print("MAPE:", mape)


R-squared: 0.7924374476545728
MAE: 0.3663232139749177
MAPE: 0.21310480913927582


In [52]:
def handle_X_set_sp(X_train_set_in):
    X_train_set = X_train_set_in.copy()
    
    y_pred_nn = neural_net.predict(X_train_set)
    y_pred_gbt = gbt.predict(X_train_set)
    y_pred_bag = bag_gbm.predict(X_train_set)
    
    #only change in function: include input vectors in training dataframe
    preds_df = pd.DataFrame(X_train_set, columns = cali_housing.feature_names)
    
    preds_df['nn'] = y_pred_nn
    preds_df['gbt'] = y_pred_gbt
    preds_df['bag'] = y_pred_bag
 
    return preds_df

def predict_from_X_set_sp(X_train_set_in):
    X_train_set = X_train_set_in.copy()

    #change final estimator's name to final_etr_sp and use handle_X_set_sp within this function
    return final_etr_sp.predict(handle_X_set_sp(X_train_set))

In [54]:
preds_df_sp = handle_X_set_sp(X_stack)
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV

param_dist = {'max_features' : ['sqrt','log2',1.0],
 'min_samples_leaf' : [1, 2, 3, 7, 11],
 'n_estimators': [50, 100],
 'oob_score': [True, False]}

pre_gs_inst_2 = RandomizedSearchCV(ExtraTreesRegressor(warm_start=True,bootstrap=True,random_state=7),
 param_distributions = param_dist,
 cv=3,
 n_iter = 15,random_state=7)

pre_gs_inst_2.fit(preds_df_sp.values, y_stack)
pre_gs_inst_2.best_params_

{'oob_score': False,
 'n_estimators': 100,
 'min_samples_leaf': 2,
 'max_features': 'log2'}

In [55]:
final_etr_sp = ExtraTreesRegressor(**{'max_features': 'log2',
 'min_samples_leaf': 2,
 'n_estimators': 3000,
 'oob_score': False,'random_state':7}).fit(preds_df_sp.values, y_stack)

In [58]:
from sklearn.model_selection import cross_val_score

cross_val_score(final_etr_sp, preds_df_sp.values, y_stack, cv=3).mean()

0.8305380660881193

In [60]:
# Assuming 'pre_gs_inst_bag' is your trained model
y_pred = pre_gs_inst_bag.predict(X_test_prin)

# Import necessary metrics
from sklearn.metrics import r2_score, mean_absolute_error

# Calculate and print R-squared
print("R-squared:", r2_score(y_test_prin, y_pred))

# Calculate and print Mean Absolute Error (MAE)
print("MAE:", mean_absolute_error(y_test_prin, y_pred))

# Calculate and print Mean Absolute Percentage Error (MAPE)
mape = (np.abs(y_test_prin - y_pred) / y_test_prin).mean()
print("MAPE:", mape)


R-squared: 0.7924374476545728
MAE: 0.3663232139749177
MAPE: 0.21310480913927582
